# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [70]:
import napari
from macrohet import dataio, tile, visualise
import os, glob
from tqdm.auto import tqdm
from skimage import io
import numpy as np

In [111]:
def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time/2:1.1f} hours"
text_size = 24
napari_scale = [1.4949402023919043e-07, 1.4949402023919043e-07]


label_text_size = 15
### glimpse size
size = 500
### resized images are to this scale
scale = 6048/1200
scale_factor = 1/5.04
from napari_animation import Animation


825530

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [71]:
%%time
expt_ID = 'ND0002'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
# metadata_fn = '/mnt/SYNO/macrohet_syno/ND0001/acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Images/Index.idx.xml'#glob.glob(os.path.join(base_dir, 'Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
# temporary hack to fix URL from incorrectly exported metadata
# metadata['URL'] = metadata.apply(dataio.generate_url, axis=1)
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 43.4 s, sys: 729 ms, total: 44.2 s
Wall time: 44.1 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0103K1F1P1R1,Ok,r01c03f01p01-ch1sk1fk1fl1.tiff,1,3,1,1,0,1,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.49+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
1,0103K1F1P1R2,Ok,r01c03f01p01-ch2sk1fk1fl1.tiff,1,3,1,1,0,2,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.723+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
2,0103K1F1P2R1,Ok,r01c03f01p02-ch1sk1fk1fl1.tiff,1,3,1,2,0,1,1,...,0,0.135468394,0,2023-11-30T17:22:10.067+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
3,0103K1F1P2R2,Ok,r01c03f01p02-ch2sk1fk1fl1.tiff,1,3,1,2,0,2,1,...,0,0.135468394,0,2023-11-30T17:22:10.287+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
4,0103K1F1P3R1,Ok,r01c03f01p03-ch1sk1fk1fl1.tiff,1,3,1,3,0,1,1,...,2E-06,0.135470405,0,2023-11-30T17:22:10.627+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388615,0612K150F9P1R2,Ok,r06c12f09p01-ch2sk150fk1fl1.tiff,6,12,9,1,149,2,1,...,-2E-06,0.1351538,268191.66,2023-12-03T20:06:16.08+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388616,0612K150F9P2R1,Ok,r06c12f09p02-ch1sk150fk1fl1.tiff,6,12,9,2,149,1,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.423+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388617,0612K150F9P2R2,Ok,r06c12f09p02-ch2sk150fk1fl1.tiff,6,12,9,2,149,2,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.657+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388618,0612K150F9P3R1,Ok,r06c12f09p03-ch1sk150fk1fl1.tiff,6,12,9,3,149,1,1,...,2E-06,0.135157794,268191.66,2023-12-03T20:06:17+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [72]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*AssayLayout.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

### Define row and column of choice

In [73]:
acq_ID = row, column = (3, 4)

### Define subset if non-square tiling or more than one contiguous region of images in imaging well. 

In [5]:
# subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

### Load images using zarr

In [10]:
import zarr

In [38]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
images = zarr.open(image_dir, mode='r')

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [124]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
images

CPU times: user 1.14 s, sys: 1.77 ms, total: 1.14 s
Wall time: 1.14 s


dask.array<max-aggregate, shape=(150, 2, 6048, 6048), dtype=uint16, chunksize=(1, 2, 2160, 2160), chunktype=numpy.ndarray>

In [125]:
%%time
images = images.compute().compute()

CPU times: user 6h 55min 49s, sys: 40min 38s, total: 7h 36min 27s
Wall time: 20min 32s


In [114]:
viewer = napari.Viewer(title = f'{expt_ID} vis with tracks')

viewer.add_image(images, channel_axis = 1, 
                 scale = napari_scale, contrast_limits=[[280, 1000],[0,3000]])
# viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'white'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

[<Image layer 'Image' at 0x7fae798d1220>,
 <Image layer 'Image [1]' at 0x7fb102c0fc40>]

### Load masks

In [208]:
fn = '/mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5'
with btrack.io.HDF5FileHandler(fn, 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation

[INFO][2024/01/17 02:41:43 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5...
[INFO][2024/01/17 02:42:03 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/01/17 02:42:03 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5


In [209]:
viewer.add_labels(segmentation, scale = napari_scale)

<Labels layer 'segmentation' at 0x7fade3b7e6d0>

### Load and overlay some tracks

In [133]:
track_dict ={}
for fn in glob.glob(os.path.join(base_dir, 'labels/testing*')):
    with btrack.io.HDF5FileHandler(fn, 'r', obj_type='obj_type_1') as reader:
        tracks = reader.tracks
    track_dict[os.path.basename(fn)] = tracks

[INFO][2024/01/17 11:58:15 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5...
[INFO][2024/01/17 11:58:15 am] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/01/17 11:58:15 am] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/01/17 11:58:15 am] Loading objects/obj_type_1 (72361, 5) (72361 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (72361, 5) (72361 filtered: None)
[INFO][2024/01/17 11:58:18 am] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4).h5
[INFO][2024/01/17 11:58:18 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4)_config_segrate_0.05_probass_0.05.json.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testi

In [197]:
# Create an animation object
animation = Animation(viewer)
viewer.dims.current_step = (0, 3023, 3023)
# Capture the initial frame
animation.capture_keyframe()
viewer.dims.current_step = (150, 3023, 3023)
# Capture the last frame (no need to change camera angles)
animation.capture_keyframe(steps =600)

# Animate to create a static video (from the first frame to the last frame)
animation.animate('/home/dayn/ND2_slow.mp4', canvas_only=True)

Rendering frames...


100%|████████████████████████████████████████████████████████| 901/901 [01:12<00:00, 12.49it/s]


In [195]:
for key in tqdm(track_dict.keys()):
    tracks = track_dict[key]
    tracks = [t for t in tracks if len(t) >15]
    napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
    viewer.add_tracks(napari_tracks, name = 'filt_'+key, scale = (napari_scale[0]/scale_factor,napari_scale[0]/scale_factor))

  0%|          | 0/12 [00:00<?, ?it/s]

In [202]:
for key in tqdm(track_dict.keys()):
    tracks = track_dict[key]
    print(key, np.mean([len(t) for t in tracks if len(t) > 50]))
    print(len(tracks))

  0%|          | 0/12 [00:00<?, ?it/s]

testing_(3, 4).h5 83.83510638297872
27889
testing_(3, 4)_config_segrate_0.05_probass_0.05.json.h5 85.0327868852459
27742
testing_(3, 4)_config_segrate_0.05_probass_0.1.json.h5 83.83510638297872
27889
testing_(3, 4)_config_segrate_0.05_probass_0.15000000000000002.json.h5 84.14285714285714
28021
testing_(3, 4)_config_segrate_0.1_probass_0.05.json.h5 85.0327868852459
27742
testing_(3, 4)_config_segrate_0.1_probass_0.1.json.h5 83.83510638297872
27889
testing_(3, 4)_config_segrate_0.1_probass_0.15000000000000002.json.h5 84.14285714285714
28021
testing_(3, 4)_config_segrate_0.15000000000000002_probass_0.05.json.h5 85.0327868852459
27742
testing_(3, 4)_config_segrate_0.15000000000000002_probass_0.1.json.h5 83.83510638297872
27889
testing_(3, 4)_config_segrate_0.15000000000000002_probass_0.15000000000000002.json.h5 84.14285714285714
28021
testing_(3, 4)_config_segrate_0.1_probass_0.010000000000000002.json.h5 84.43715846994536
27631
testing_(3, 4)_config_segrate_0.1_probass_0.9.json.h5 82.69430

In [212]:
# Create an animation object
animation = Animation(viewer)
viewer.dims.current_step = (0, 3023, 3023)
# Capture the initial frame
animation.capture_keyframe()
viewer.dims.current_step = (150, 3023, 3023)
# Capture the last frame (no need to change camera angles)
animation.capture_keyframe(steps =900)

# Animate to create a static video (from the first frame to the last frame)
animation.animate('/home/dayn/ND2_slow_w_ID_tracks.mp4', canvas_only=True)

Rendering frames...


100%|████████████████████████████████████████████████████████| 901/901 [01:22<00:00, 10.94it/s]


### Focusing on one set of tracks

In [178]:
fn = '/mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4)_config_segrate_0.05_probass_0.05.json.h5'
with btrack.io.HDF5FileHandler(fn, 'r', obj_type='obj_type_1') as reader:
    seg = reader.segmentation
    tracks = reader.tracks
recolored_segmentation = btrack.utils.update_segmentation(reader.segmentation, reader.tracks ,scale = (1/scale_factor,1/scale_factor))


[INFO][2024/01/17 12:40:02 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4)_config_segrate_0.1_probass_0.1.json.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4)_config_segrate_0.1_probass_0.1.json.h5...
[INFO][2024/01/17 12:40:48 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/01/17 12:40:49 pm] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/01/17 12:40:49 pm] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/01/17 12:40:49 pm] Loading objects/obj_type_1 (72361, 5) (72361 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (72361, 5) (72361 filtered: None)
[INFO][2024/01/17 12:40:50 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_(3, 4)_config_segrate_0.1_probass_0.1.json.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/testing_

In [186]:
tracks = [t for t in tracks if len(t) >15]
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
viewer.add_tracks(napari_tracks, name = 'filt'+key, scale = (napari_scale[0]/scale_factor,napari_scale[0]/scale_factor))


<Labels layer 'recolored_segmentation' at 0x7fad8de544f0>

In [188]:
napari_scale

[1.4949402023919043e-07, 1.4949402023919043e-07]

In [189]:
viewer.add_labels(recolored_segmentation, scale = napari_scale)

<Labels layer 'recolored_segmentation' at 0x7fadaa9d0ac0>

# Testing best set of tracks

In [191]:
for key in tqdm(track_dict.keys()):
    tracks = track_dict[key]
    tracks = [t for t in tracks if len(t) >15]
    napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
    viewer.add_tracks(napari_tracks, name = 'filt_'+key, scale = (napari_scale[0]/scale_factor,napari_scale[0]/scale_factor))

  0%|          | 0/12 [00:00<?, ?it/s]